### 전국횡단보도표준데이터
* 녹색 신호가 가장 긴 값의 주소 출력
* 시별 횡단보도 갯수 구하기
* 녹색 신호시간 평균 및 적색 신호 시간 평균
### 내가한거

### 전국횡단보도표준데이터
1. 횡단보도 연장, 녹색신호시간 : 상관도
2. 자전거 횡단도 겸용 비율(전체 대비) 
3. 차로수별 자전거 횡단도 카운트/비율
4. 차로수별 보행자 신호등 유무 카운트/비율
5. 차로수별 음향신호기설치 유무 카운트/비율

### 화면 출력
1. 상관도 수치
2. 비율 수치
3. 카운트, 비율
4. 카운트, 비율
5. 카운트, 비율

In [1]:
import requests
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

np.set_printoptions(precision=5, suppress=True)

In [11]:
def my_split(s):
    block_start = False
    start_index = 0
    ret_list=[]
    for i, c in enumerate(s):
        if block_start==False:
            if c==',':
                ret_list.append(s[start_index:i])
                start_index=i+1
            elif c=='"':
                block_start=True
                start_index = i
        else:
            if c=='"':
                block_start=False
    if s[-1]!=',':
        ret_list.append(s[start_index:])
    return ret_list

def split_len(data_list):
    len_list = []
    for e in data_list:
        len_list.append(len(e))
    print(set(len_list))
    if len (set(len_list)) > 1:
        for i in set(len_list):
            print(i, len_list.count(i))

In [16]:
# 데이터 정재
raw_data = []
with open('전국횡단보도표준데이터.csv') as f:
    for line in f:
        raw_data.append(my_split(line))
split_len(raw_data)
np_data = np.array(raw_data[1:])
print(np_data)

{28}
[['경기도' '남양주시' '강변북로' ... '2023-01-31' '3990000' '경기도 남양주시\n']
 ['경기도' '남양주시' '강변북로' ... '2023-01-31' '3990000' '경기도 남양주시\n']
 ['경기도' '남양주시' '강변북로' ... '2023-01-31' '3990000' '경기도 남양주시\n']
 ...
 ['대전광역시' '대덕구' '대화로' ... '2023-01-05' '6300000' '대전광역시\n']
 ['대전광역시' '대덕구' '대화로' ... '2023-01-05' '6300000' '대전광역시\n']
 ['대전광역시' '서구' '둔산북로' ... '2023-01-05' '6300000' '대전광역시\n']]


In [17]:
for i in enumerate(raw_data[0]):
    print(i)

(0, '시도명')
(1, '시군구명')
(2, '도로명')
(3, '소재지도로명주소')
(4, '소재지지번주소')
(5, '횡단보도관리번호')
(6, '횡단보도종류')
(7, '자전거횡단도겸용여부')
(8, '고원식적용여부')
(9, '위도')
(10, '경도')
(11, '차로수')
(12, '횡단보도폭')
(13, '횡단보도연장')
(14, '보행자신호등유무')
(15, '보행자작동신호기유무')
(16, '음향신호기설치여부')
(17, '녹색신호시간')
(18, '적색신호시간')
(19, '교통섬유무')
(20, '보도턱낮춤여부')
(21, '점자블록유무')
(22, '집중조명시설유무')
(23, '관리기관명')
(24, '관리기관전화번호')
(25, '데이터기준일자')
(26, '제공기관코드')
(27, '제공기관명\n')


In [68]:
# 횡단보도 연장, 녹색신호시간 : 상관도
extend_list = np_data[:, 13].astype(np.float64)
green_list = np_data[:, 17]
green_list = np.where(green_list == '',0.0 , green_list)
green_list = green_list.astype(np.float64)

corr_value = np.corrcoef(extend_list, green_list)
print(corr_value[0, 1])


0.15847542321039498


In [75]:
# 자전거 횡단도 겸용 비율(전체 대비)
filter1 = np_data[: , 7] == 'Y'
bike_list = np_data[filter1]
bike_persent = len(bike_list)*100 / len(np_data)
print(bike_persent)

9.228


In [83]:
# 차로수별 자전거 횡단도 카운트/비율
persent = []
yes_bike_road_count = bike_list[:, 11]
road, count = np.unique(yes_bike_road_count, return_counts=True)
print(road)
print(count)
for i in count:
    persent.append(i * 100 / sum(count))
persent = np.array(persent)
print(persent)

['1' '10' '11' '12' '13' '2' '3' '4' '5' '6' '7' '8' '9']
[ 423   39   12    7    3  859  489 1002  525  593  371  166  125]
[ 9.16775  0.84525  0.26008  0.15171  0.06502 18.61725 10.59818 21.71651
 11.37841 12.85219  8.04075  3.59775  2.70915]


In [108]:
# 차로수별 보행자 신호등 유무 카운트/비율
sub_data = np_data[:, [11, 15]]
filter1 = sub_data[:,1] != ' '
sub_data = sub_data[filter1]
print(sub_data)

pass1 = dict()
for cnt, ymd in sub_data:
#     print(cnt, ymd)
    if cnt in pass1.keys():
        pass1[cnt].append(ymd)
    else:
        pass1[cnt] = [ymd]
# print(pass1)

pass2 = []
for cnt in pass1:
    val, count = np.unique(pass1[cnt], return_counts=True)
    y, ratio = 0, 0
    if 'Y' in 


[['3' 'Y']
 ['9' 'Y']
 ['9' 'Y']
 ...
 ['4' 'Y']
 ['3' 'Y']
 ['3' 'Y']]
3
9
1
4
8
2
11
10
5
6
7
0
